In [1]:
import h5py

file = './processedData/fastMRI_4_cart.h5'
with h5py.File(file, 'r') as F:
     coil = F['trnCsm'][:]
     mask = F['trnMask'][:]

In [2]:
import torch
from models import mri

def get_max_eig(coil, mask):
        r""" compute maximal eigenvalue

        References:
            * Beck A, Teboulle M.
              A Fast Iterative Shrinkage-Thresholding Algorithm for Linear Inverse Problems.
              SIAM J Imaging Sci (2009). DOI: https://doi.org/10.1137/080716542
            * Tan Z, Hohage T, Kalentev O, Joseph AA, Wang X, Voit D, Merboldt KD, Frahm J.
              An eigenvalue approach for the automatic scaling of unknowns in model-based reconstructions: Application to real-time phase-contrast flow MRI.
              NMR Biomed (2017). DOI: https://doi.org/10.1002/nbm.3835
        """
        A = mri.SenseOp(coil, mask)

        # x = torch.randn(size=A.shape, dtype=coil.dtype)
        coil = torch.from_numpy(coil)
        x = torch.randn_like(coil)
        x = torch.swapaxes(x, 0, 1)
        for _ in range(30):
            y = A.adj(A.fwd(x))
            max_eig = torch.linalg.norm(y).ravel()
            x = y / max_eig

            # print(max_eig)

        return max_eig

In [3]:
get_max_eig(coil, mask)

tensor([0.6702])